In [1]:
import requests

# note that CLIENT_ID refers to 'personal use script' and SECRET_TOKEN to 'token'
auth = requests.auth.HTTPBasicAuth('jIr-nVzfumFqGw', 'ZHqDRjAawvPJKdHsfUwrpS57LDtiBQ')

# here we pass our login method (password), username, and password
data = {'grant_type': 'password',
        'username': 'jpz129',
        'password': 'IloveRhiannon129!@('}

# setup our header info, which gives reddit a brief description of our app
headers = {'User-Agent': 'MyBot/0.0.1'}

# send our request for an OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)

# convert response to JSON and pull access_token value
TOKEN = res.json()['access_token']

# add authorization to our headers dictionary
headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}

# while the token is valid (~2 hours) we just add headers=headers to our requests
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)

<Response [200]>

In [2]:
res = requests.get("https://oauth.reddit.com/r/seafood/hot",
                   headers=headers)

print(res.json())  # let's see what we get

{'kind': 'Listing', 'data': {'modhash': None, 'dist': 25, 'children': [{'kind': 't3', 'data': {'approved_at_utc': None, 'subreddit': 'Seafood', 'selftext': '', 'author_fullname': 't2_4hpdb', 'saved': False, 'mod_reason_title': None, 'gilded': 0, 'clicked': False, 'title': 'Scallops', 'link_flair_richtext': [], 'subreddit_name_prefixed': 'r/Seafood', 'hidden': False, 'pwls': 6, 'link_flair_css_class': None, 'downs': 0, 'thumbnail_height': 140, 'top_awarded_type': None, 'hide_score': False, 'name': 't3_m4yqca', 'quarantine': False, 'link_flair_text_color': 'dark', 'upvote_ratio': 0.98, 'author_flair_background_color': None, 'subreddit_type': 'public', 'ups': 42, 'total_awards_received': 0, 'media_embed': {}, 'thumbnail_width': 140, 'author_flair_template_id': None, 'is_original_content': False, 'user_reports': [], 'secure_media': None, 'is_reddit_media_domain': True, 'is_meta': False, 'category': None, 'secure_media_embed': {}, 'link_flair_text': None, 'can_mod_post': False, 'score': 42,

In [6]:
import pandas as pd
# make a request for the trending posts in /r/Python
res = requests.get("https://oauth.reddit.com/r/seafood",
                   headers=headers)

df = pd.DataFrame()  # initialize dataframe

# loop through each post retrieved from GET request
for post in res.json()['data']['children']:
    # append relevant data to dataframe
    df = df.append({
        'subreddit': post['data']['subreddit'],
        'title': post['data']['title'],
        'selftext': post['data']['selftext'],
        'upvote_ratio': post['data']['upvote_ratio'],
        'ups': post['data']['ups'],
        'downs': post['data']['downs'],
        'score': post['data']['score']
    }, ignore_index=True)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   downs         25 non-null     float64
 1   score         25 non-null     float64
 2   selftext      25 non-null     object 
 3   subreddit     25 non-null     object 
 4   title         25 non-null     object 
 5   ups           25 non-null     float64
 6   upvote_ratio  25 non-null     float64
dtypes: float64(4), object(3)
memory usage: 1.5+ KB


In [8]:
df.head()

,downs,score,selftext,subreddit,title,ups,upvote_ratio
0,0.0,38.0,,Seafood,Scallops,38.0,0.95
1,0.0,5.0,,Seafood,Buttermilk fried sea scallops for my fathers b...,5.0,0.86
2,0.0,3.0,,Seafood,How To Make | Maryland Style | Crab Cakes | Wi...,3.0,1.00
3,0.0,60.0,,Seafood,Steamed mussels with white wine and garlic,60.0,1.00
4,0.0,8.0,,Seafood,[Homemade] Shrimp Stir Fry,8.0,0.85
